In [1]:
import json
from urllib.parse import urlparse, parse_qs, urljoin

import requests as rq
from bs4 import BeautifulSoup


In [2]:
def get_daylywebtoons():

    webtoon_main_url = WC['TOP_URL']
    res = rq.get(webtoon_main_url)
    main_soup = BeautifulSoup(res.content, 'lxml')
    webtoon_links = [{"title": a_tag.get('title'), "link": urljoin(WC['TOP_URL'], a_tag.get('href'))}
                      for a_tag in main_soup.select('#content a.title')]
    
    return webtoon_links


In [3]:
WC = {'TOP_URL' : 'http://comic.naver.com/webtoon/weekday.nhn'}


In [4]:
webtoons = get_daylywebtoons()
webtoons[:5]


[{'link': 'http://comic.naver.com/webtoon/list.nhn?titleId=183559&weekday=mon',
  'title': '신의 탑'},
 {'link': 'http://comic.naver.com/webtoon/list.nhn?titleId=648419&weekday=mon',
  'title': '뷰티풀 군바리'},
 {'link': 'http://comic.naver.com/webtoon/list.nhn?titleId=602910&weekday=mon',
  'title': '윈드브레이커'},
 {'link': 'http://comic.naver.com/webtoon/list.nhn?titleId=654774&weekday=mon',
  'title': '소녀의 세계'},
 {'link': 'http://comic.naver.com/webtoon/list.nhn?titleId=679519&weekday=mon',
  'title': '대학일기'}]

In [5]:
webtoons[1]

{'link': 'http://comic.naver.com/webtoon/list.nhn?titleId=648419&weekday=mon',
 'title': '뷰티풀 군바리'}

In [5]:
def get_qs(url, key):
    url_query = urlparse(url).query
    query_dict = parse_qs(url_query)
    value = query_dict[key][0]
    return value


In [6]:
def get_webtoon_id(number):
    webtoon_url = webtoons[number]['link']
    webtoon_title = webtoons[number]['title']
    print('webtoon_title :', webtoon_title)

    webtoon_id = get_qs(webtoon_url, 'titleId')
    print('webtoon_id : ', webtoon_id)
    weekday = get_qs(webtoon_url, 'weekday')
    print('weekday : ', weekday)
    
    return webtoon_id


In [7]:
def get_webtoon_url(number):
    
    webtoon_url = webtoons[number]['link']
    print('webtoon_url :', webtoon_url)
    
    return webtoon_url


In [8]:
def get_start_chapter(n):
    url = get_webtoon_url(n)
    res = rq.get(url)
    startchapter = BeautifulSoup(res.content, 'lxml')
    aa = startchapter.select('#content > table > tr > td.title > a')[0]
    u = aa.get('href')
    url_query = urlparse(u).query
    query_dict = parse_qs(url_query)
    start_chapter = query_dict['no'][0]
    return start_chapter


In [9]:
def get_comment_soup(webtoon_id, chapter, comment_page=1):
    Referer_url='http://m.comic.naver.com/scrolltoon/comment.nhn?titleId='+str(webtoon_id)+'&no='+str(chapter)
    coment_url='http://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=comic&templateId=webtoon&pool=cbox3&_callback=jQuery33104420385964332538_1524620976824&lang=ko&country=KR&objectId='+str(webtoon_id) + '_' + str(chapter)+'&categoryId=&pageSize=15&indexSize=10&groupId=&listType=OBJECT&pageType=more&page='+str(comment_page)+'&initialize=true&userType=&useAltSort=true&replyPageSize=10&sort=NEW&_=1524620976825'
    header = {
            "Host": "apis.naver.com",
            "Referer": Referer_url,
        }

    res = rq.get(coment_url, headers = header)
    soup = BeautifulSoup(res.content, 'lxml')
    return soup


In [10]:
def from_soup_to_maxlpage(tt):
    content_text = tt.select('p')[0].text
    one = content_text.find('(') + 1
    two = content_text.find(');')
    content = json.loads(content_text[one:two])
    max_comment_page = content['result']['pageModel']['totalPages']
    return max_comment_page


In [11]:
def get_comment_ls(soup, chapter, comment_ls):
    content_text = soup.select('p')[0].text
    one = content_text.find('(') + 1
    two = content_text.find(');')
    content = json.loads(content_text[one:two])
    comments = content['result']['commentList']

    for n in range(0,14+1):
        ls = []
        com = comments[n]['contents']
        userid = comments[n]['maskedUserName'] + '('+comments[n]['maskedUserId'] + ')'
        like = comments[n]['sympathyCount']
        dislike = comments[n]['antipathyCount']
        ls = [chapter, userid, com, like, dislike]
        comment_ls.append(ls)
        
    return comment_ls


In [12]:
def get_data(title, webtoon_id, chapter):

    tt = get_comment_soup(webtoon_id, chapter, comment_page=1)
    max_comment_page= from_soup_to_maxlpage(tt)
    comment_ls = []
    
    for comment_page in range(1,max_comment_page+1):      
        soup = get_comment_soup(webtoon_id, chapter, comment_page)
        try:
            get_comment_ls(soup, chapter, comment_ls)
        except :
            pass
        
        
    print('chapter '+str(chapter)+' Load finish')
            
    return comment_ls


In [22]:
webtoons[100]

{'link': 'http://comic.naver.com/webtoon/list.nhn?titleId=703837&weekday=thu',
 'title': '아기낳는만화'}